<a href="https://colab.research.google.com/github/s4m-y4k/ConvinInternAssignment/blob/main/ConvinAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install django

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 5.8 MB/s eta 0:00:00


In [10]:
pip install djangorestframework

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00


In [3]:
from django.shortcuts import redirect
from django.views import View
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow

In [4]:
class GoogleCalendarInitView(View):
  def get(self, request):
    scopes = ['https://www.googleapis.com/auth/calendar.readonly']
    credentails = Credentials.from_authorized_user_info(info=None)
    flow = Flow.from_client_config(
        client_config={
            'web': {
                    'client_id': 'YOUR_CLIENT_ID',
                    'client_secret': 'YOUR_CLIENT_SECRET',
                    'redirect_uris': ['http://localhost:8000/rest/v1/calendar/redirect/'],
                    'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
                    'token_uri': 'https://oauth2.googleapis.com/token',
                    'access_type': 'offline',
                    'prompt': 'consent'
                }
            },
            scopes=scopes,
            redirect_uri='http://localhost:8000/rest/v1/calendar/redirect/'
        )
    authorization_url, state = flow.authorization_url(access_type='offline')
    return redirect(authorization_url)

In [13]:
import os
from django.conf import settings

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Untitled4.settings')
settings.configure()

In [14]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from rest_framework.views import APIView

class GoogleCalendarInitView(APIView):
    def get(self, request):
        flow = InstalledAppFlow.from_client_secrets_file(
            '/content/drive/MyDrive/ConvinAssignment/client_secret_678416271905-4uhkqflr21htlh7dav6s20g0titltka6.apps.googleusercontent.com.json',
            scopes=['https://www.googleapis.com/auth/calendar.events.readonly']
        )
        auth_url, _ = flow.authorization_url(prompt='consent')
        return HttpResponseRedirect(auth_url)

In [15]:
from rest_framework.views import APIView
from rest_framework.response import Response
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow

class GoogleCalendarRedirectView(APIView):
    def get(self, request):
        scopes = ['https://www.googleapis.com/auth/calendar.readonly']


        credentials = Credentials.from_authorized_user_info(info=None)

        flow = Flow.from_client_config(
            client_config={
                'web': {
                    'client_id': 'YOUR_CLIENT_ID',
                    'client_secret': 'YOUR_CLIENT_SECRET',
                    'redirect_uris': ['http://localhost:8000/rest/v1/calendar/redirect/'],
                    'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
                    'token_uri': 'https://oauth2.googleapis.com/token',
                    'access_type': 'offline',
                    'prompt': 'consent'
                }
            },
            scopes=scopes,
            redirect_uri='http://localhost:8000/rest/v1/calendar/redirect/'
        )
        authorization_response = request.build_absolute_uri()
        flow.fetch_token(authorization_response=authorization_response)
        credentials = flow.credentials
        return Response({'message': 'Events retrieved successfully!'})

In [17]:
from googleapiclient.discovery import build

class GoogleCalendarRedirectView(APIView):
    def get(self, request):
        flow = InstalledAppFlow.from_client_secrets_file(
            '/content/drive/MyDrive/ConvinAssignment/client_secret_678416271905-4uhkqflr21htlh7dav6s20g0titltka6.apps.googleusercontent.com.json',
            scopes=['https://www.googleapis.com/auth/calendar.events.readonly']
        )
        flow.fetch_token(authorization_response=request.build_absolute_uri())
        credentials = flow.credentials
        service = build('calendar', 'v3', credentials=credentials)
        events = service.events().list(calendarId='CalendarID').execute()
        return Response(events)